In [ ]:
%pip install langchain-core langchain-openai langchain-exa

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "..."
os.environ["EXA_API_KEY"] = "..."

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_PROJECT"] = "erick-default"

In [7]:
from langchain_exa import ExaSearchRetriever

retriever = ExaSearchRetriever(k=3, highlights=True)
documents = retriever.invoke("best time to visit japan")

In [9]:

print(documents[0].metadata["highlights"])

['It\x92s a cluster of public holidays: Shōwa Day (29 April), Constitution Day (3 May), Greenery Day (4 May) and Children\x92s Day (5 May) that sees many Japanese people leaving the cities to visit their families. Trains are full, hotel prices rocket, and a lot of restaurants and shops close in the cities. Joining in with the festivities can be a nice experience, but generally speaking Golden Week is not the best time to travel in Japan. Late autumn to March and high summer (July and August) tend to be the cheapest times to visit Japan.']


In [10]:

print(documents[0].metadata["url"])

https://www.responsibletravel.com/holidays/japan/travel-guide/best-time-to-visit-japan


In [12]:
from langchain_exa import ExaSearchRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda


retriever = ExaSearchRetriever(k=3, highlights=True)

document_prompt = PromptTemplate.from_template("""
<source>
    <url>{url}</url>
    <highlights>{highlights}</highlights>
</source>
""")

document_chain = RunnableLambda(
    lambda document: {
        "highlights": document.metadata["highlights"], 
        "url": document.metadata["url"]
    }
) | document_prompt

retrieval_chain = retriever | document_chain.map() | (lambda docs: "\n".join([i.text for i in docs]))

print(retrieval_chain.invoke("best time to visit japan"))


<source>
    <url>https://www.responsibletravel.com/holidays/japan/travel-guide/best-time-to-visit-japan</url>
    <highlights>['It\x92s a cluster of public holidays: Shōwa Day (29 April), Constitution Day (3 May), Greenery Day (4 May) and Children\x92s Day (5 May) that sees many Japanese people leaving the cities to visit their families. Trains are full, hotel prices rocket, and a lot of restaurants and shops close in the cities. Joining in with the festivities can be a nice experience, but generally speaking Golden Week is not the best time to travel in Japan. Late autumn to March and high summer (July and August) tend to be the cheapest times to visit Japan.']</highlights>
</source>


<source>
    <url>https://alljapantours.com/japan/travel/when-to-go/The-Best-Time-to-Visit-Japan/</url>
    <highlights>['Without a doubt, it’s the most romantic time of year and attracts millions of visitors from all over the world. And that’s not an exaggeration. The iconic cherry blossom season in 

In [13]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

generation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert research assistant. You use xml-formatted context to research people's questions."),
    ("human", """
Please answer the following query based on the provided context. Please cite your sources at the end of your response.:
     
Query: {query}
---
<context>
{context}
</context>
""")
])

llm = ChatOpenAI()

chain = RunnableParallel({
    "query": RunnablePassthrough(),
    "context": retrieval_chain,
}) | generation_prompt | llm

In [14]:
chain.invoke("best time to visit japan")

AIMessage(content='The best time to visit Japan depends on several factors, including weather, festivals, and personal preferences. However, there are a few key periods that are considered popular times to visit.\n\nOne of the most popular times to visit Japan is during the cherry blossom season, which typically occurs from late March to early April. This is a romantic and beautiful time of year when cities like Tokyo and Kyoto are adorned with pink cherry blossoms. Millions of visitors from around the world come to Japan to witness this natural spectacle.\n\nAnother popular time to visit Japan is in late autumn to March. During this time, the weather is generally cool and the tourist crowds are smaller. This period is considered off-peak, which means that prices for accommodation and attractions tend to be lower. It is a great time to explore Japan\'s cultural attractions without the crowds.\n\nHigh summer, specifically July and August, is another time that tends to be cheaper to visi